<a href="https://colab.research.google.com/github/Gregory-Wong/Healthcare-Model-analysis/blob/main/Naive_BayesLinear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#setting up initial packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB


In [ ]:
#load in dataset
from google.colab import files
import io
uploaded = files.upload()

Saving linear.csv to linear.csv


In [ ]:
#create/view dataframe
df = pd.read_csv(io.BytesIO(uploaded['linear.csv']))
df.head()

,Year,Data_Value,Data_Value_Alt,Confidence_Limit_Low,Confidence_Limit_High,Break_out
0,2011,1.6,1.6,1.1,2.2,Female
1,2011,1.7,1.7,1.4,2.0,Female
2,2011,1.7,1.7,1.4,2.1,Female
3,2011,1.8,1.8,1.5,2.1,Female
4,2011,1.8,1.8,1.6,2.2,Female


**Data Cleaning**

In [ ]:
#convert to string
df['Break_out'] = df['Break_out'].map({
    'Male' :int(0),
    'Female' :int(1),
    'Elderly Group' :int(2),
    'Hispanic' :int(3),
    'Middle Group' :int(4),
    'Non-Hispanic Asian' :int(5),
    'Non-Hispanic Black' :int(6),
    'Non-Hispanic White' :int(7),
    'Other' :int(8),
    'Overall' :int(9),
    'Young group' :int(10),
    'Young Group' :int(10)})

df


,Year,Data_Value,Data_Value_Alt,Confidence_Limit_Low,Confidence_Limit_High,Break_out
0,2011,1.6,1.6,1.1,2.2,1
1,2011,1.7,1.7,1.4,2.0,1
2,2011,1.7,1.7,1.4,2.1,1
3,2011,1.8,1.8,1.5,2.1,1
4,2011,1.8,1.8,1.6,2.2,1
...,...,...,...,...,...,...
85790,2015,97.7,97.7,97.0,98.3,10
85791,2015,97.7,97.7,97.2,98.1,10
85792,2015,97.8,97.8,97.3,98.2,10
85793,2015,98.1,98.1,97.3,98.7,10


In [ ]:
#check for null values
df.isna().sum()

Year                         0
Data_Value               28768
Data_Value_Alt               0
Confidence_Limit_Low     30162
Confidence_Limit_High    30162
Break_out                    0
dtype: int64

In [ ]:
#Fill Na with mean of the column
#Confidence Limit High
df['Confidence_Limit_High'] = df['Confidence_Limit_High'].fillna(df['Confidence_Limit_High'].mean())

#Confidence Limit Low
df['Confidence_Limit_Low'] = df['Confidence_Limit_Low'].fillna(df['Confidence_Limit_Low'].mean())

#Data Value
df['Data_Value'] = df['Data_Value'].fillna(df['Data_Value'].mean())

In [ ]:
#check for null values again
df.isna().sum()

Year                     0
Data_Value               0
Data_Value_Alt           0
Confidence_Limit_Low     0
Confidence_Limit_High    0
Break_out                0
dtype: int64

In [ ]:
#look at data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85795 entries, 0 to 85794
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Year                   85795 non-null  int64  
 1   Data_Value             85795 non-null  float64
 2   Data_Value_Alt         85795 non-null  float64
 3   Confidence_Limit_Low   85795 non-null  float64
 4   Confidence_Limit_High  85795 non-null  float64
 5   Break_out              85795 non-null  int64  
dtypes: float64(4), int64(2)
memory usage: 3.9 MB


In [ ]:
#converty float to in
df['Data_Value_Alt'] = df['Data_Value_Alt'].astype(int)
df['Data_Value'] = df['Data_Value'].astype(int)
df['Confidence_Limit_Low'] = df['Confidence_Limit_Low'].astype(int)
df['Confidence_Limit_High'] = df['Confidence_Limit_High'].astype(int)

In [ ]:
#Check data type agin
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85795 entries, 0 to 85794
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   Year                   85795 non-null  int64
 1   Data_Value             85795 non-null  int64
 2   Data_Value_Alt         85795 non-null  int64
 3   Confidence_Limit_Low   85795 non-null  int64
 4   Confidence_Limit_High  85795 non-null  int64
 5   Break_out              85795 non-null  int64
dtypes: int64(6)
memory usage: 3.9 MB


In [ ]:
#stat breakdown
df.describe()

,Year,Data_Value,Data_Value_Alt,Confidence_Limit_Low,Confidence_Limit_High,Break_out
count,85795.000000,85795.000000,85795.000000,85795.000000,85795.000000,85795.000000
mean,2013.000117,33.073198,21.484096,30.032496,36.275529,5.272452
std,1.414181,23.439054,28.590378,22.505048,23.755623,3.319036
min,2011.000000,0.000000,-2.000000,0.000000,0.000000,0.000000
25%,2012.000000,16.000000,-1.000000,14.000000,20.000000,2.000000
50%,2013.000000,33.000000,8.000000,30.000000,36.000000,5.000000
75%,2014.000000,33.000000,33.000000,30.000000,37.000000,8.000000
max,2015.000000,99.000000,99.000000,99.000000,99.000000,10.000000


In [ ]:
#Split data set
x = df.drop(['Data_Value'], axis = 1)
y = df.pop('Data_Value')

In [ ]:
#Split the dataset
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=1)

Navie Bayes Model

In [ ]:
#scale the data 
#string to float caucasian
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
#preform LDA
NB_model = GaussianNB()

# fit the model
NB_model.fit(x_train, y_train)

y_predict_NB = NB_model.predict(x_test)

# model score
predict_train_NB = NB_model.predict(x_train)
predict_test_NB = NB_model.predict(x_test)

# accuracy score
NB_train_score = NB_model.score(x_train,y_train)
NB_test_score = NB_model.score(x_test,y_test)

# f1-score
NB_f1_score = metrics.f1_score(y_test,predict_test_NB, average=None)
NB_recall = metrics.recall_score(y_test, predict_test_NB, average=None)

print('Accuracy on Train set',NB_train_score)
print('Accuracy on Test set',NB_test_score)
print('F1-score on Test set:',NB_f1_score)
print(metrics.classification_report(y_test, predict_test_NB))

Accuracy on Train set 0.992190622086053
Accuracy on Test set 0.9912972531955399
F1-score on Test set: [0.83018868 0.97832817 0.98933902 0.9893438  0.98028552 0.98231827
 0.99143207 0.98919568 0.98412698 0.97662338 0.98998569 0.99365079
 0.9955157  0.99095023 0.98492462 0.99481865 0.99270073 0.9800995
 0.98525074 0.99164345 0.97391304 0.97852029 0.98617512 0.99775281
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         0.98748883 1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.         1.         1.
 1.         1.         1.         1.     